# 트랜스포머를 이용한 한국어 챗봇

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import tensorflow_datasets as tfds
import tensorflow as tf

## 데이터 로드

In [8]:
train_data = pd.read_csv('C:/workspace/Data/ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
print('챗봇 샘플의 개수: {}'.format(len(train_data)))

챗봇 샘플의 개수: 11823


In [10]:
# Missing Value 확인
train_data.isna().sum()

Q        0
A        0
label    0
dtype: int64

- 형태소 분석기를 사용하지 않고, 다른 방법인 학습 기반의 토크나이저를 사용
- 그래서 원 데이터에서 ?, ., ! 와 같은 구두점을 미리 처리해두어야 하는데, 구두점들을 단순히 제거할 수도 있지만, 여기서는 구두점 앞에 공백. 즉, 띄어쓰기를 추가하여 다른 문자들과 구분

In [15]:
questions = []

for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! >> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [16]:
answers = []

for sentence in train_data['A']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! >> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [17]:
# 구두점들이 띄어쓰기를 통해 분리되었는지 확인

print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


## 단어 집합 생성
- 서브워드텍스트인코더 사용
    - 학습데이터로부터 학습하여 서브워드로 구성된 단어집합을 생성한다

In [19]:
# 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어집합 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size = 2**13)

- 인코더-디코더 모델계열에는 디코더의 입력으로 사용할 시작을 의미하는 SOS와 종료토큰 EOS 또한 존재한다. 해당 토큰들도 단어 집합에 포함시킬 필요가 있으므로 이 두 토큰에 정수를 부여해주자

In [ ]:
# 시작토큰과 종료토큰에 대한 정수부여
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작토큰과 종료토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2